In [4]:
from utilities import files
import os.path as op
from os import sep
import nibabel as nb
import numpy as np
import matplotlib.pylab as plt
from matplotlib import cm, colors
from mne import read_epochs, pick_types
import pandas as pd
from scipy.spatial.distance import euclidean
from tools import transform_atlas, fsavg_vals_to_native
import new_files
import json
from mne.time_frequency import psd_array_multitaper, psd_array_welch

from fooof.sim.gen import gen_aperiodic
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import trimesh
import open3d as o3d
from joblib import Parallel, delayed

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
def compute_csd(surf_tcs, times, mean_dist, n_surfs):
    # Compute CSD
    nd=1;
    spacing=mean_dist*10**-3

    csd=np.zeros((n_surfs, surf_tcs.shape[1]))
    for t in range(surf_tcs.shape[1]):
        phi=surf_tcs[:,t]
        csd[0,t]=surf_tcs[0,t]
        csd[1,t]=surf_tcs[1,t]
        for z in range(2,n_surfs-3):
            csd[z,t]=(phi[z+2]-2*phi[z]+phi[z-2])/((nd*spacing)**2)
        csd[-2,t]=surf_tcs[-2,t]
        csd[-1,t]=surf_tcs[-1,t]            
    
    return csd


def smooth_csd(csd, n_surfs):
    # interpolate CSD in space
    y = np.linspace(0,n_surfs-1,n_surfs)
    Yi=np.linspace(0,n_surfs-1,500)
    
    f=interp1d(y,csd,kind='cubic',axis=0)
    csd_smooth=f(Yi)
    
    csd_smooth=savgol_filter(csd_smooth, 51, 3, axis=1)
    
    return csd_smooth


def compute_rel_power(power, freqs):
    power = gaussian_filter(power, sigma=[1.5, 2])

    if np.min(power[:]) < 0:
        power = power - np.min(power[:])
    rel_power = np.zeros(power.shape)
    for freq in range(len(freqs)):
        rel_power[:, freq] = (power[:, freq] - np.min(power[:, freq])) / (
                    np.max(power[:, freq]) - np.min(power[:, freq]))

    return rel_power


def get_crossover(freqs,rel_per_power,rel_aper_power):
    n_chans=rel_per_power.shape[0]
    ab_idx = np.where((freqs >= 7) & (freqs <= 30))[0]
    g_idx = np.where((freqs >= 50) & (freqs <= 125))[0]
    ab_rel_pow = np.mean(rel_per_power[:, ab_idx], axis=1)
    g_rel_pow = np.mean(rel_aper_power[:, g_idx], axis=1)
    crossovers = detect_crossing_points(ab_rel_pow, g_rel_pow)
    assert(len(crossovers)<=2)
    if len(crossovers) > 1:
        dist1 = np.min([crossovers[0], n_chans - crossovers[0]])
        dist2 = np.min([crossovers[1], n_chans - crossovers[1]])
        if dist1 > dist2:
            crossover = crossovers[0]
        else:
            crossover = crossovers[1]
    else:
        crossover = crossovers[0]
    return crossover


def detect_crossing_points(ab_rel_pow, g_rel_pow):
    crossing_points = []

    # Iterate through the series
    for i in range(1, len(ab_rel_pow)):
        # Check if the series cross each other
        if (ab_rel_pow[i] > g_rel_pow[i] and ab_rel_pow[i - 1] < g_rel_pow[i - 1]) or \
                (ab_rel_pow[i] < g_rel_pow[i] and ab_rel_pow[i - 1] > g_rel_pow[i - 1]):
            crossing_points.append(i)

    return crossing_points


def all_layers_ROI_map(layer_len, n_surf, ROI_indexes):
    return np.array([i[ROI_indexes] for i in np.split(np.arange(layer_len*n_surf), n_surf)]).flatten()


def fooofinator_par(freqs, psd, f_lims, n_jobs=-1):
    start_params=np.arange(f_lims[0],f_lims[1]-5,1)
    
    def run_fooof(i):
        start=start_params[i]
        fg=FOOOF(aperiodic_mode='fixed')
        fg.fit(freqs,psd, [start,f_lims[1]])
        if fg.has_model:
            ap_params=fg.get_params('aperiodic_params')
            return gen_aperiodic(freqs, ap_params)
        else:
            return np.zeros(psd.shape)*np.nan
    aperiodic = Parallel(
        n_jobs=n_jobs
    )(delayed(run_fooof)(i) for i in range(len(start_params)))
    return np.nanmedian(np.array(aperiodic),axis=0)


def fooofinator(freqs, psd, f_lims):
    start_params=np.arange(f_lims[0],f_lims[1]-5,1)
    test_aperiodic=[]
    for i,start in enumerate(start_params):
        fg=FOOOF(aperiodic_mode='fixed')
        fg.fit(freqs,psd, [start,f_lims[1]])
        if fg.has_model:
            ap_params=fg.get_params('aperiodic_params')
            test_aperiodic.append(gen_aperiodic(freqs, ap_params))
    return np.median(np.array(test_aperiodic),axis=0)

In [6]:
raw_path = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01"
inverted_path = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/inverted"

In [7]:
annot_paths = [
    "/home/mszul/git/DANC_multilayer_laminar/assets/lh.HCPMMP1.annot",
    "/home/mszul/git/DANC_multilayer_laminar/assets/rh.HCPMMP1.annot"
]
fsavg_sphere_paths = [
    "/home/mszul/git/DANC_multilayer_laminar/assets/lh.sphere.reg.gii",
    "/home/mszul/git/DANC_multilayer_laminar/assets/rh.sphere.reg.gii"
]
fsnat_sphere_paths = [
    "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/lh.sphere.reg.gii",
    "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/rh.sphere.reg.gii"
]
pial_path = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.gii"
pial_ds_path = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.ds.gii"
pial_ds_nodeep = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.ds.link_vector.nodeep.gii"
pial_ds_inflated = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/fs/surf/pial.ds.inflated.nodeep.gii"
glasser = "/home/mszul/git/DANC_multilayer_laminar/assets/atlas_glasser_2016.csv"

In [8]:
n_surf = 11
atlas = pd.read_csv(glasser)
ml_nodeep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.nodeep.gii");
vertices, faces, normals = ml_nodeep.agg_data()
vertices = np.split(vertices, n_surf, axis=0)
faces = np.split(faces, n_surf, axis=0)
normals = np.split(normals, n_surf, axis=0)
layer_len = vertices[0].shape[0]
nufs = new_files.Files()

In [9]:
inflated_nodeep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/pial.ds.inflated.nodeep.gii")
i_vertices, i_faces = inflated_nodeep.agg_data()

In [10]:
lab_col_map_path = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/glasser2006_labels_per_vertex.npy"
mesh_colors_path = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/glasser2006_colours_per_vertex.npy"
cortical_thickness_path = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/cortical_thickness_vertex.npy"
bigbrain_boundaries_list_path = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/big_brain_boundaries.json"


if not all([op.exists(lab_col_map_path), op.exists(mesh_colors_path)]):
    mesh_colors, lab_col_map = transform_atlas(annot_paths, fsavg_sphere_paths, fsnat_sphere_paths, pial_path, pial_ds_path, pial_ds_nodeep)
    np.save(lab_col_map_path, lab_col_map)
    np.save(mesh_colors_path, mesh_colors)
elif all([op.exists(lab_col_map_path), op.exists(mesh_colors_path)]):
    mesh_colors = np.load(mesh_colors_path)
    lab_col_map = np.load(lab_col_map_path)

if not all([op.exists(cortical_thickness_path)]):
    cortical_thickness = np.array([euclidean(vertices[0][vx], vertices[-1][vx]) for vx in range(vertices[0].shape[0])])
    np.save(cortical_thickness_path, cortical_thickness)
elif all([op.exists(cortical_thickness_path)]):
    cortical_thickness = np.load(cortical_thickness_path)


if not all([op.exists(bigbrain_boundaries_list_path)]):
    bigbrain_l_paths = nufs.get_files("/home/mszul/git/DANC_multilayer_laminar/assets/big_brain_layer_thickness", "*.gii", strings=["hemi-L"], prefix="tpl-fsaverage")
    bigbrain_r_paths = nufs.get_files("/home/mszul/git/DANC_multilayer_laminar/assets/big_brain_layer_thickness", "*.gii", strings=["hemi-R"], prefix="tpl-fsaverage")
    bigbrain_lr_paths = list(zip(thicc_l_paths, thicc_r_paths))
    layers_fsaverage_values = {i+1: [nb.load(i).agg_data() for i in bigbrain_lr_paths[i]] for i in range(len(bigbrain_lr_paths))}
    layers_fsnative_ds_values = {i: fsavg_vals_to_native(
        layers_fsaverage_values[i],
        fsavg_sphere_paths,
        fsnat_sphere_paths, 
        pial_path, 
        pial_ds_path, 
        pial_ds_nodeep
    ) for i in layers_fsaverage_values.keys()}

    overall_thickness = np.sum(layers, axis=0)
    bigbrain_boundaries_prop = {i: np.divide(
        layers_fsnative_ds_values[i], 
        overall_thickness, 
        where=overall_thickness != 0
    ) for i in layers_fsnative_ds_values.keys()}

    np.save(
        lab_col_map_path,
        lab_col_map
    )
    np.save(
        mesh_colors_path,
        mesh_colors
    )
    np.save(
        cortical_thickness_path,
        cortical_thickness
    )

    bigbrain_boundaries_list = {i: list(bigbrain_boundaries_prop[i].astype(float)) for i in bigbrain_boundaries_prop.keys()}
    with open(bigbrain_boundaries_list_path, "w") as fp:
        json.dump(bigbrain_boundaries_list, fp, indent=4)

elif all([op.exists(bigbrain_boundaries_list_path)]):
    with open(bigbrain_boundaries_list_path, "r") as fp:
        bigbrain_boundaries_list = json.load(fp)
    bigbrain_boundaries_list = {i: np.array(bigbrain_boundaries_list[i]) for i in bigbrain_boundaries_list.keys()}

In [11]:
ROI_maps = {i: [i == j.decode("utf=8") for j in lab_col_map] for i in atlas.USED_LABEL.values}
ROI_indexes = {i: np.arange(layer_len)[ROI_maps[i]] for i in ROI_maps.keys()}

In [12]:
target_ROI = atlas.loc[(atlas.PRIMARY_SECTION == 6) & (atlas.HEMISPHERE == "L")].USED_LABEL.tolist()
print(target_ROI)

['L_4_ROI', 'L_3b_ROI', 'L_1_ROI', 'L_2_ROI', 'L_3a_ROI']


In [13]:
ROI_combined_map = np.hstack([ROI_indexes[i] for i in target_ROI])
ROI_combined_map.sort()

In [14]:
ROI_combined_map = np.random.choice(ROI_combined_map, size=30)

In [15]:
info = {
    "visual": (-0.2, 0.8),
    "motor": (-0.5, 0.5)
}

times = {}
crop_times = {}
n_surf = 11

In [16]:
fif_paths = files.get_files(raw_path, "autoreject", "epo.fif")[2]
MU_paths = files.get_files(inverted_path, "MU", ".tsv")[2]
fif_MU = list(zip(fif_paths, MU_paths))

In [17]:
fif, MU = fif_MU[0]
core_name = fif.split(sep)[-1].split(".")[0].replace("autoreject-", "")
epo_type = [i for i in info.keys() if i in core_name][0]
crop_time = info[epo_type]
fif = read_epochs(fif, verbose=False)
fif = fif.pick_types(meg=True, ref_meg=False, misc=False)
times[epo_type] = fif.times
sfreq = fif.info["sfreq"]
fif = fif.get_data()
MU = pd.read_csv(MU, sep="\t", header=None).to_numpy()

In [18]:
multilayer_vert = np.split(np.arange(layer_len *n_surf), n_surf)
ROI_verts = np.hstack([i[ROI_combined_map] for i in multilayer_vert])
MU_roi = MU[ROI_verts]
source = []
for trial in fif:
    trial_source = np.dot(trial.T, MU_roi.T).T
    trial_source = np.split(trial_source, n_surf, axis=0)
    trial_source = np.array(trial_source) # layer x vertex x time
    source.append(trial_source)
source = np.array(source) # trial x layer x vertex x time

In [ ]:
# psd_per_layer = []
# periodic_per_layer = []
# aperiodic_per_layer = []
# fooofed_spec_per_layer = []
# for surf in range(n_surf):
#     winsize = int(sfreq)
#     overlap = int(winsize/2)   
#     psd,freqs = psd_array_welch(
#         source[:, surf, :, :], sfreq,
#         fmin=0.1, fmax=125, n_fft=2000,
#         n_overlap=overlap, n_per_seg=winsize, 
#         window="hann", verbose=False         
#     )
#     mean_psd = np.nanmean(psd,axis=0)
#     psd_per_layer.append(mean_psd)
#     fg = FOOOFGroup(aperiodic_mode="fixed")
#     fg.fit(freqs, mean_psd, [0.1,125])
#     ap_vx = 10 ** np.array([fg.get_fooof(i)._ap_fit for i in range(mean_psd.shape[0])]) # results stored in linear scale
#     ff_spec_vx = 10 ** np.array([fg.get_fooof(i)._ap_fit for i in range(mean_psd.shape[0])]) # results stored in linear scale
#     pr_vx = mean_psd - ap_vx
#     aperiodic_per_layer.append(ap_vx)
#     fooofed_spec_per_layer.append(ff_spec_vx)
#     periodic_per_layer.append(pr_vx)

# vx_r = range(psd_per_layer.shape[1])
# psd_per_layer = np.array(psd_per_layer) # layer x vertex x freqs
# periodic_per_layer = np.array(periodic_per_layer) # layer x vertex x freqs
# aperiodic_per_layer = np.array(aperiodic_per_layer) # layer x vertex x freqs
# fooofed_spec_per_layer = np.array(fooofed_spec_per_layer) # layer x vertex x freqs
# psd_rel_power = np.stack([compute_rel_power(psd_per_layer[:,i,:], freqs) for i in vx_r], axis=1)
# periodic_rel_power = np.stack([compute_rel_power(periodic_per_layer[:,i,:], freqs) for i in vx_r], axis=1)
# aperiodic_rel_power = np.stack([compute_rel_power(aperiodic_per_layer[:,i,:], freqs) for i in vx_r], axis=1)
# fooofed_rel_power = np.stack([compute_rel_power(fooofed_spec_per_layer[:,i,:], freqs) for i in vx_r], axis=1)
# crossover = []
# for i in vx_r:
#     try:
#         c = get_crossover(freqs, periodic_rel_power[:,i,:], aperiodic_rel_power[:,i,:])
#     except:
#         c = np.nan
#     crossover.append(c)
# crossover = np.array(crossover)

In [ ]:
vx_r = range(source.shape[2])
psd_per_layer = []
periodic_per_layer = []
aperiodic_per_layer = []
flims = [0.1,125]
for surf in range(n_surf):
    winsize = int(sfreq)
    overlap = int(winsize/2)   
    psd,freqs = psd_array_welch(
        source[:, surf, :, :], sfreq,
        fmin=flims[0], fmax=flims[1], n_fft=2000,
        n_overlap=overlap, n_per_seg=winsize, 
        window="hann", verbose=False         
    )
    mean_psd = np.nanmean(psd,axis=0)
    aperiodic = np.vstack([fooofinator_par(freqs, i, flims, n_jobs=-1) for i in mean_psd])
    mean_psd = np.log10(mean_psd)
    psd_per_layer.append(mean_psd)
    periodic = mean_psd - aperiodic
    periodic_per_layer.append(periodic)
    aperiodic_per_layer.append(aperiodic)
    print(surf)

psd_per_layer = np.array(psd_per_layer) # layer x vertex x freqs
periodic_per_layer = np.array(periodic_per_layer) # layer x vertex x freqs
aperiodic_per_layer = np.array(aperiodic_per_layer) # layer x vertex x freqs
psd_rel_power = np.stack([compute_rel_power(psd_per_layer[:,i,:], freqs) for i in vx_r], axis=1)
periodic_rel_power = np.stack([compute_rel_power(periodic_per_layer[:,i,:], freqs) for i in vx_r], axis=1)
aperiodic_rel_power = np.stack([compute_rel_power(aperiodic_per_layer[:,i,:], freqs) for i in vx_r], axis=1)


0


In [ ]:
crossover = []
for i in vx_r:
    try:
        c = get_crossover(freqs, periodic_rel_power[:,i,:], aperiodic_rel_power[:,i,:])
    except:
        c = np.nan
    crossover.append(c)
crossover = np.array(crossover)

In [ ]:
crossover

In [ ]:
crop_ix = np.where((times[epo_type] >= crop_time[0]) & (times[epo_type] <= crop_time[-1]))[0]
source_mean = np.mean(source[:,:,:,crop_ix], axis=0)
crop_times[epo_type] = times[epo_type][crop_ix]

In [ ]:
vertices_range = np.arange(crossover.shape[0])
vx_crossover = vertices_range[~np.isnan(crossover)]
vx_no_crossover = vertices_range[np.isnan(crossover)]

In [ ]:
layer_labels = ["I", "II", "III", "IV", "V", "VI"]

vertex = vx_crossover[1]
print(vertex)
cov = crossover[vertex]/10
thickness_vx = cortical_thickness[ROI_combined_map][vertex]
mean_thickness_vx = thickness_vx/n_surf
csd = compute_csd(source_mean[:, vertex, :], crop_times[epo_type], mean_thickness_vx, n_surf)
csd_smooth = smooth_csd(csd, n_surf)
f, ax = plt.subplots(1,4, figsize=(16,4))
ax[0].set_title("Aperiodic Spectrum- relative power")
ax[0].imshow(aperiodic_rel_power[:,vertex,:], aspect="auto", extent=[freqs[0], freqs[-1], 1, 0])
ax[0].set_xlabel('Frequency (Hz)')
ax[0].set_ylabel('Depth as a proportion of\nthe overall cortical thickness')
ax[0].axvline(7, linestyle="dashed", lw=1, c="white")
ax[0].axvline(30, linestyle="dashed", lw=1, c="white")
ax[0].axvline(50, linestyle="dashed", lw=1, c="white")
ax[0].axhline(cov, lw=1, c="white")
# ax[0].set_ylim(1,0)
ax[1].set_title("Periodic Spectrum- relative power")
ax[1].imshow(periodic_rel_power[:,vertex,:], aspect="auto", extent=[freqs[0], freqs[-1], 1, 0])
ax[1].set_xlabel('Frequency (Hz)')
ax[1].axvline(7, linestyle="dashed", lw=1, c="white")
ax[1].axvline(30, linestyle="dashed", lw=1, c="white")
ax[1].axvline(50, linestyle="dashed", lw=1, c="white")
# ax[1].set_ylim(1,0)
ax[1].axhline(cov, lw=1, c="white")
ab_idx = np.where((freqs >= 7) & (freqs <= 30))[0]
g_idx = np.where((freqs >= 50) & (freqs <= 125))[0]
ab_rel_pow = np.mean(periodic_rel_power[:, vertex, ab_idx], axis=1)
g_rel_pow = np.mean(aperiodic_rel_power[:, vertex, g_idx], axis=1)
ax[2].set_title("Alpha-Beta vs Gamma crossover")
ax[2].plot(ab_rel_pow,np.arange(periodic_rel_power.shape[0])/10,label='apha-beta')
ax[2].plot(g_rel_pow,np.arange(periodic_rel_power.shape[0])/10,label='gamma')
ax[2].set_xlabel('Relative power')
ax[2].legend()
ax[2].set_ylim(1,0)
ax[2].axhline(cov, lw=1, c="red")
vertex_val = [bigbrain_boundaries_list[i][ROI_combined_map[vertex]] for i in bigbrain_boundaries_list.keys()]
ROI_mean = [np.mean(bigbrain_boundaries_list[i][ROI_combined_map]) for i in bigbrain_boundaries_list.keys()]
ROI_std = [np.std(bigbrain_boundaries_list[i][ROI_combined_map]) for i in bigbrain_boundaries_list.keys()]
max_smooth= np.max(np.abs(csd_smooth))
divnorm = colors.TwoSlopeNorm(vmin=-max_smooth, vcenter=0, vmax=max_smooth)
ax[3].set_title("Current Source Density")
im = ax[3].imshow(
    csd_smooth, norm=divnorm, origin="lower",
    aspect="auto",
    extent=[
        crop_times[epo_type][0], 
        crop_times[epo_type][-1], 
        1, 0
    ],
    cmap="RdBu_r"
)
ax[3].set_ylim(1,0)
cs = plt.cm.afmhot(np.linspace(0, 1, num=11))
ax[3].axhline(cov, lw=1, c="red")
for l_ix, th in enumerate(np.cumsum(ROI_mean)):
    ax[3].axhline(th, linestyle=(0, (5,5)), c="black", lw=0.5)
    ax[3].axhspan(th-ROI_std[l_ix], th+ROI_std[l_ix], alpha=0.05, color="black", lw=0)
    ax[3].annotate(layer_labels[l_ix],[-0.49, th-0.01],size=15)
plt.tight_layout()

f, ax = plt.subplots(1,1, figsize=(10,5))
cr = plt.cm.cool(np.linspace(0, 1, num=11))
for c_i, c in enumerate(cr):
    ax.plot(freqs, psd_per_layer[c_i,vertex,:] , lw=1, alpha=1, c=c)
    ax.plot(freqs, aperiodic_per_layer[c_i,vertex,:] , lw=1, alpha=1, c=c)
ax.set_xlim(0,60)
ax.axvline(7, linestyle="dashed", lw=1, c="black")
ax.axvline(30, linestyle="dashed", lw=1, c="black")
ax.axvline(50, linestyle="dashed", lw=1, c="black")

In [ ]:
# vertex_val = [bigbrain_boundaries_list[i][ROI_combined_map][vertex] for i in bigbrain_boundaries_list.keys()]
# bigbrain_boundaries_list["1"][ROI_combined_map]
bigbrain_boundaries_list["1"]

In [ ]:
cs = plt.cm.viridis(np.linspace(0, 1, num=10))
values = cs[:,:3]
gray = np.array([0.4, 0.4, 0.4])
nan_c = np.array([1., 1., 1.])

In [ ]:
colour_matrix = np.repeat(gray.reshape(1,-1), layer_len, axis=0)
for c_ix, c in enumerate(values):
    c_ix = c_ix+1
    colour_matrix[ROI_indexes["L_4_ROI"][crossover == c_ix]] = c
colour_matrix[ROI_indexes["L_4_ROI"][np.isnan(crossover)]] = nan_c

In [ ]:
mesh = trimesh.Trimesh(vertices=i_vertices, faces=i_faces, process=False, validate=False)
mesh = mesh.as_open3d
mesh.vertex_colors = o3d.utility.Vector3dVector(colour_matrix)
mesh.compute_vertex_normals(normalized=True)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
fz = FOOOF()
fz.fit(freqs, psd_per_layer[0,1,:],[0,125])
ffntr = fooofinator(freqs, psd_per_layer[0,1,:], [0.1,125])
f, ax = plt.subplots(1, 1, figsize=(10,5))
ax.plot(freqs, np.log10(psd_per_layer[0,1,:]), label="PSD")
ax.plot(freqs, fz._ap_fit, label="FOOOF")
ax.plot(freqs, ffntr, label="FOOOFINATOR")

ax.legend()

In [ ]:
mean_p